In [1]:
""" 

Solves the discrete logarithm problem 
g^x = y (mod p) - (1) 
i.e. finds the value of x that satisfies the congruence. p is a prime number.

Suppose x = mq + r where m is floor(sqrt(p)) integer between 0 and p.
=> 0 <= r < m
=> g^(mq + r) = y(mod N) - (2)
=> g^mq . g^r = y(mod N) - (3)
=> g^mq = y / g^r (mod N) -(4)

Choose a particular m.
Calculate RHS of eqn (4)  from r=0 to r=m-1 since r can only take values from 0 to m-1.
Evaluate the LHS of eqn (4) for integer values of q. If the congruence in eqn(1) holds then the congruence in eqn(4) will 
also hold for some value(s) of r and q.
Hence x = r + mq


Now coming to the reason as to why m is chosen as floor(sqrt(p)) is that the LHS of eqn(4) takes 
m operations evaluating g^r for m values. g^mq has to be evaluated for values of q until mq < p.  This implies q < p/m

So the total number of operations is m + p/m. This function is minimized if m = sqrt(p)
"""

' \n\nSolves the discrete logarithm problem \ng^x = y (mod p) - (1) \ni.e. finds the value of x that satisfies the congruence. p is a prime number.\n\nSuppose x = mq + r where m is floor(sqrt(p)) integer between 0 and p.\n=> 0 <= r < m\n=> g^(mq + r) = y(mod N) - (2)\n=> g^mq . g^r = y(mod N) - (3)\n=> g^mq = y / g^r (mod N) -(4)\n\nChoose a particular m.\nCalculate RHS of eqn (4)  from r=0 to r=m-1 since r can only take values from 0 to m-1.\nEvaluate the LHS of eqn (4) for integer values of q. If the congruence in eqn(1) holds then the congruence in eqn(4) will \nalso hold for some value(s) of r and q.\nHence x = r + mq\n\n\nNow coming to the reason as to why m is chosen as floor(sqrt(p)) is that the LHS of eqn(4) takes \nm operations evaluating g^r for m values. g^mq has to be evaluated for values of q until mq < p.  This implies q < p/m\n\nSo the total number of operations is m + p/m. This function is minimized if m = sqrt(p)\n'

In [2]:
import math
import gmpy2 as gp
import sys
import timeit

In [3]:
# Helper functions

def evaluate_y_by_g_pow_r(y,g,p,m):
    hash_table = {}
    g_inv = gp.invert(g,p)
    for r in range(0,m):
        g_inv_r = gp.powmod(g_inv,r,p)
        lhs = gp.mod(gp.mul(y,g_inv_r),p)
        
        hash_table[lhs] = r
    return hash_table
def compare_with_hashtable(hash_table,g,m,p):
    g_pow_m = gp.powmod(g,m,p)
    for q in range(0,m):
        rhs = gp.powmod(g_pow_m,q,p)
        if rhs in hash_table:
            return q,hash_table[rhs]
    return None,None

In [4]:
def baby_step_giant_step(g,y,p,x_rng):
    if(p<=0):
        raise ValueError("p should be a positive integer")
    if g <= 0:
        raise ValueError("g should be a positive integer")
    if y <= 0:
        raise ValueError("y should be a positive integer")
    if x_rng<=0:
        raise ValueError("x_rng should be a positive integer")
    if gp.gcd(g,p)!=1:
        raise ValueError("Discrete Log does not exist.")
        
    g = g % p
    y = y % p
    m = int(math.sqrt(x_rng))
    # Evaluate y/g^r for r varying from 0 to m-1
    print("Building Hash Table...")
    hash_table = evaluate_y_by_g_pow_r(y,g,p,m)
    print("Comparing with Hash Table...")
    q,r = compare_with_hashtable(hash_table,g,m,p)
    if q==None:
        return None
    else:
        x = m*q+r
        return x    

In [5]:
import time
start = time.time()
x = baby_step_giant_step(g=11717829880366207009516117596335367088558084999998952205599979459063929499736583746670572176471460312928594829675428279466566527115212748467589894601965568
                             ,y=3239475104050450443565264378728065788649097520952449527834792452971981976143292558073856937958553180532878928001494706097394108577585732452307673444020333
                             ,p=13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006084171
                             ,x_rng = 2**40)
end = time.time()
print("Time taken: "+str(end-start))
print(x)

Building Hash Table...
Comparing with Hash Table...
Time taken: 14.741785764694214
375374217830
